# Validation and comparisons different LLMs

## Imports and environment setup

In [ ]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

import numpy as np
from datetime import datetime
import pickle

from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go
from sklearn.manifold import TSNE

from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd



In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


## Costants

In [ ]:
DATA_PATH='/content/drive/MyDrive/SanRaffaele/Data/Dataset LLM'

In [ ]:
# Mappa tipi da tabella
ENTITY_TYPE_MAP = {
    "n_cartella": "Number",
    "data_ingresso_cch": "Date",
    "data_dimissione_cch": "Date",
    "nome": "Text",
    "cognome": "Text",
    "sesso": "Categorical_MF",
    "numero di telefono": "Text",
    "età al momento dell'intervento": "Number",
    "data_di_nascita": "Date",
    "Diagnosi": "Text",
    "Anamnesi": "Text",
    "Motivo ricovero": "Text",
    "classe_nyha": "Categorical_1234",
    "angor": "Boolean",
    "STEMI/NSTEMI": "Boolean",
    "scompenso_cardiaco_nei_3_mesi_precedenti": "Boolean",
    "fumo": "Categorical_012",
    "diabete": "Boolean",
    "ipertensione": "Boolean",
    "dislipidemia": "Boolean",
    "BPCO": "Boolean",
    "stroke_pregresso": "Boolean",
    "TIA_pregresso": "Boolean",
    "vasculopatiaperif": "Boolean",
    "neoplasia_pregressa": "Boolean",
    "irradiazionetoracica": "Boolean",
    "insufficienza_renale_cronica": "Boolean",
    "familiarita_cardiovascolare": "Boolean",
    "limitazione_mobilita": "Boolean",
    "endocardite": "Boolean",
    "ritmo_all_ingresso": "Categorical_012",
    "fibrillazione_atriale": "Categorical_012",
    "dialisi": "Boolean",
    "elettivo_urgenza_emergenza": "Categorical_012",
    "pm": "Boolean",
    "crt": "Boolean",
    "icd": "Boolean",
    "pci_pregressa": "Boolean",
    "REDO": "Boolean",
    "Anno REDO": "Date",
    "Tipo di REDO": "Text",
    "Terapia": "Text",
    "lasix": "Boolean",
    "lasix_dosaggio": "Number",
    "nitrati": "Boolean",
    "antiaggregante": "Boolean",
    "dapt": "Boolean",
    "anticoagorali": "Boolean",
    "aceinib": "Boolean",
    "betabloc": "Boolean",
    "sartanici": "Boolean",
    "caantag": "Boolean",
    "esami_all_ingresso": "Text",
    "Decorso_post_operatorio": "Text",
    "IABP/ECMO/IMPELLA": "Boolean",
    "Inotropi": "Boolean",
    "secondo_intervento": "Boolean",
    "Tipo_secondo_intervento": "Text",
    "II_Run": "Boolean",
    "Causa_II_Run_CEC": "Text",
    "LCOS": "Boolean",
    "Impianto_PM_post_intervento": "Boolean",
    "Stroke_TIA_post_op": "Boolean",
    "Necessità_di_trasfusioni": "Boolean",
    "IRA": "Boolean",
    "Insufficienza_respiratoria": "Boolean",
    "FA_di_nuova_insorgenza": "Boolean",
    "Ritmo_alla_dimissione": "Categorical_012",
    "H_Stay_giorni (da intervento a dimissione)": "Number",
    "Morte": "Boolean",
    "Causa_morte": "Text",
    "data_morte": "Date",
    "esami_alla_dimissione": "Text",
    "terapia_alla_dimissione": "Text"
}


## Functions

In [ ]:
def create_dict_model_dataframe(list_json_response, list_entità):
    '''
    crea una lista di dataframe nome modello:dataframe con le risposte di un modello
    per tutti i file
    '''
    rows = []
    #print(len(list_json_response))

    #if(len(list_json_response) == 2):
      #list_json_response = list_json_response[1]

    #print(len(list_json_response))

    for response in list_json_response:

        # Converti la risposta in dizionario {entity: value}
        row_dict = dict(zip(response.iloc[:, 0], response.iloc[:, 1]))
        rows.append(row_dict)

    # Crea DataFrame finale
    df = pd.DataFrame(rows)

    # Ordina le colonne per mantenerle in base a ENTITY_TYPE_MAP
    df = df[[col for col in ENTITY_TYPE_MAP.keys() if col in df.columns]]

    for col in df.columns:
        for i, row in df.iterrows():
            if row[col] in ['NAN', 'nan', '', 'NaN']:
                df.at[i, col] = None


    return df


In [ ]:
def show_comparison(dict_df):
    fig = go.Figure()

    fig.update_layout(
        title_text='Comparazione numero di entità trovate',
        xaxis_title_text='Entità',
        yaxis_title_text='Count',
        bargap=0.2,
        bargroupgap=0.1
    )

    # Determina tutte le entità da usare sull'asse X
    all_entities = set()
    all_entities = sorted(list(ENTITY_TYPE_MAP.keys()))

    fig.update_xaxes(categoryorder='array', categoryarray=all_entities)

    for name, df in dict_df.items():
        counts = []
        for ent in all_entities:
            # Use notnull().sum() to count non-null values
            count = df[ent].notnull().sum() if ent in df.columns else 0
            counts.append(count)

        fig.add_trace(go.Bar(name=name, x=all_entities, y=counts))

    fig.show()

In [ ]:
def validate_value(value, expected_type):
    if pd.isnull(value):
        return True
    try:
        if expected_type == "Number":
            float(value)
        elif expected_type == "Date":
            pd.to_datetime(value, errors='raise', dayfirst=True)
        elif expected_type == "Text":
            str(value)
        elif expected_type == "Boolean":
            if str(value).strip().lower() not in {"true", "false", "0", "1", "vero", "falso"}:
                return False
        elif expected_type == "Categorical_MF":
            if str(value).strip().upper() not in {"M", "F"}:
                return False
        elif expected_type == "Categorical_012":
            if str(value).strip() not in {"0", "1", "2"}:
                return False
        elif expected_type == "Categorical_1234":
            if str(value).strip() not in {"1", "2", "3", "4"}:
                return False
        else:
            return False
        return True
    except:
        return False


In [ ]:
def score_entities(df):
    df = df.copy()

    for ent in df.columns:
        expected_type = ENTITY_TYPE_MAP.get(ent)
        score_col = f"{ent}_score"
        df[score_col] = 0  # default

        for i, val in df[ent].items():
            if val is None:
                continue
            elif expected_type is None:
                df.at[i, score_col] = -2  # entità sconosciuta
            elif not validate_value(val, expected_type):
                df.at[i, score_col] = -1  # tipo non valido
            # altrimenti lascia 0 come punteggio positivo

    return df


In [ ]:
def print_info_entità(df):
    # Identifica le colonne score
    score_cols = [col for col in df.columns if col.endswith('_score')]
    entity_cols = [col.replace('_score', '') for col in score_cols]

    total_found = 0
    total_wrong = 0
    total_hallucinated = 0
    entity_found_counts = {col: 0 for col in entity_cols}

    for i, row in df.iterrows():
        found_entities = [col for col in entity_cols if pd.notnull(row[col])]


        wrong_type_entities = [
            (col, row[col]) for col in entity_cols
            if row.get(f"{col}_score") == -1
        ]

        hallucinated_entities = [
            (col, row[col]) for col in entity_cols
            if row.get(f"{col}_score") == -2
        ]

        merged_score = len(wrong_type_entities) + len(hallucinated_entities) * 2

        total_found += len(found_entities)
        total_wrong += len(wrong_type_entities)
        total_hallucinated += len(hallucinated_entities)

        for col in found_entities:
            entity_found_counts[col] += 1

        print(f"\n--- Riga {i+1} ---")
        print(f"Numero di entità totali: {len(entity_cols)}")
        print(f"Numero di entità trovate: {len(found_entities)}")
        print(f"Entità trovate: {found_entities}")
        print(f"Score coerenza valori: {merged_score}")
        print(f"Entità con valore sbagliato: {wrong_type_entities}")
        print(f"Entità inventate: {hallucinated_entities}")

    num_rows = len(df)
    num_entities_total = len(entity_cols) * num_rows

    print("\n=== Statistiche Finali ===")
    print(f"Media per riga:")
    print(f"- Entità trovate: {total_found / num_rows:.2f}")
    print(f"- Entità sbagliate: {total_wrong / num_rows:.2f}")
    print(f"- Entità allucinate: {total_hallucinated / num_rows:.2f}")

    print(f"\nPercentuali rispetto al totale atteso di entità ({num_entities_total}):")
    print(f"- % Entità trovate: {100 * total_found / num_entities_total:.2f}%")
    print(f"- % Entità sbagliate: {100 * total_wrong / num_entities_total:.2f}%")
    print(f"- % Entità allucinate: {100 * total_hallucinated / num_entities_total:.2f}%")

    print("\n--- Numero medio di entità trovate per tipo ---")
    entity_avg_df = pd.DataFrame({
        "Entità": list(entity_found_counts.keys()),
        "Media entità trovate": [count / num_rows for count in entity_found_counts.values()]
    })
    print(entity_avg_df.to_string(index=False))


In [ ]:
def show_entity_distribution_multiple(df_dict):
    all_data = []
    stats = []
    color_palette = px.colors.qualitative.Plotly
    model_colors = {}

    for i, (model_name, df) in enumerate(df_dict.items()):
        score_cols = [col for col in df.columns if col.endswith('_score')]
        entity_cols = [col.replace('_score', '') for col in score_cols]
        found_entities_per_row = df[entity_cols].notnull().sum(axis=1)

        temp_df = pd.DataFrame({
            "Entità trovate": found_entities_per_row,
            "Modello": model_name
        })
        all_data.append(temp_df)

        # Statistiche
        mean = found_entities_per_row.mean()
        std = found_entities_per_row.std()
        stats.append((model_name, mean, std))

        # Assegna colore a modello
        model_colors[model_name] = color_palette[i % len(color_palette)]

    # Unione dati
    final_df = pd.concat(all_data, ignore_index=True)

    # Plot principale
    fig = px.histogram(
        final_df,
        x="Entità trovate",
        color="Modello",
        barmode="overlay",
        nbins=max(final_df["Entità trovate"]) + 1,
        title="Distribuzione entità trovate per paziente con Media e Std",
        labels={"Entità trovate": "Numero di entità trovate"},
        color_discrete_map=model_colors
    )

    # Aggiunge le linee di media
    for model_name, mean, std in stats:
        fig.add_vline(
            x=mean,
            line_width=2,
            line_dash="dash",
            line_color=model_colors[model_name],
            annotation_text=f"{model_name}<br>μ={mean:.1f}, σ={std:.1f}",
            annotation_position="top",
            annotation_font_size=12
        )

    fig.update_layout(
        bargap=0.1,
        xaxis=dict(dtick=1),
        yaxis_title="Numero di pazienti",
    )

    fig.show()

In [ ]:
def show_hallucinated_entities(dict_df):
    fig = go.Figure()

    fig.update_layout(
        title_text='Comparazione entità allucinate',
        xaxis_title_text='Entità',
        yaxis_title_text='Count',
        bargap=0.2,
        bargroupgap=0.1
    )

    # Determina tutte le entità da usare sull'asse X
    all_entities = set()
    all_entities = list(ENTITY_TYPE_MAP.keys())

    fig.update_xaxes(categoryorder='array', categoryarray=all_entities)
    for name, df in dict_df.items():
        counts = []
        for ent in all_entities:
            count = (df[f'{ent}_score'] == -2).sum() if f'{ent}_score' in df.columns else 0
            counts.append(count)

        fig.add_trace(go.Bar(name=name, x=all_entities, y=counts))

    fig.show()


In [ ]:
def show_wrong_type_entities(dict_df):
    fig = go.Figure()

    fig.update_layout(
        title_text='Comparazione entità con il tipo sbagliato',
        xaxis_title_text='Entità',
        yaxis_title_text='Count',
        bargap=0.2,
        bargroupgap=0.1
    )

    # Determina tutte le entità da usare sull'asse X
    all_entities = set()
    all_entities = list(ENTITY_TYPE_MAP.keys())

    fig.update_xaxes(categoryorder='array', categoryarray=all_entities)
    for name, df in dict_df.items():
        counts = []
        for ent in all_entities:
            count = (df[f'{ent}_score'] == -1).sum() if f'{ent}_score' in df.columns else 0
            counts.append(count)

        fig.add_trace(go.Bar(name=name, x=all_entities, y=counts))

    fig.show()

## Comparisons

In [ ]:
# Creo un dizionario modello risposta
list_file=os.listdir(DATA_PATH)
list_file

dict_models_json_response={}
for file in list_file[:-2]:
  file_path=os.path.join(DATA_PATH,file)
  with open(file_path, 'rb') as f:
    list_json_response = pickle.load(f)
    dict_models_json_response[file]=list_json_response
print(dict_models_json_response.keys())

dict_keys(['DeepSeek-V3.pkl', 'LLM_Llama-3.3-70B.pkl'])


In [ ]:
dict_models_df_response={}
for k,v in dict_models_json_response.items():
  response_df=create_dict_model_dataframe(v,ENTITY_TYPE_MAP.keys())
  response_df=score_entities(response_df)
  dict_models_df_response[k]=response_df

print(dict_models_df_response.keys())

dict_keys(['DeepSeek-V3.pkl', 'LLM_Llama-3.3-70B.pkl'])


In [ ]:
for k,v in dict_models_df_response.items():
  print(k)
  print(v.shape)

DeepSeek-V3.pkl
(291, 148)
LLM_Llama-3.3-70B.pkl
(291, 148)


In [ ]:
pd.set_option('display.max_rows', None)

I compare the None values; as we can see, most of the time only a small number of entities are found compared to the total number of entities.

In [ ]:
fig = px.imshow(dict_models_df_response['LLM_Llama-3.3-70B.pkl'].isnull())
fig.show()

In [ ]:
fig = px.imshow(dict_models_df_response['DeepSeek-V3.pkl'].isnull())
fig.show()

In [ ]:
print_info_entità(dict_models_df_response['DeepSeek-V3.pkl'])


--- Riga 1 ---
Numero di entità totali: 74
Numero di entità trovate: 36
Entità trovate: ['n_cartella', 'data_ingresso_cch', 'data_dimissione_cch', 'nome', 'cognome', 'sesso', 'numero di telefono', 'data_di_nascita', 'Diagnosi', 'Anamnesi', 'Motivo ricovero', 'classe_nyha', 'angor', 'fumo', 'diabete', 'dislipidemia', 'TIA_pregresso', 'vasculopatiaperif', 'insufficienza_renale_cronica', 'familiarita_cardiovascolare', 'REDO', 'Anno REDO', 'Tipo di REDO', 'Terapia', 'lasix', 'lasix_dosaggio', 'antiaggregante', 'dapt', 'betabloc', 'caantag', 'Decorso_post_operatorio', 'Inotropi', 'FA_di_nuova_insorgenza', 'Ritmo_alla_dimissione', 'esami_alla_dimissione', 'terapia_alla_dimissione']
Score coerenza valori: 1
Entità con valore sbagliato: [('classe_nyha', 'II')]
Entità inventate: []

--- Riga 2 ---
Numero di entità totali: 74
Numero di entità trovate: 18
Entità trovate: ['n_cartella', 'data_ingresso_cch', 'data_dimissione_cch', 'nome', 'cognome', 'numero di telefono', 'data_di_nascita', 'Diagno

In [ ]:
print_info_entità(dict_models_df_response['LLM_Llama-3.3-70B.pkl'])


--- Riga 1 ---
Numero di entità totali: 74
Numero di entità trovate: 17
Entità trovate: ['n_cartella', 'data_ingresso_cch', 'data_dimissione_cch', 'nome', 'cognome', 'numero di telefono', 'data_di_nascita', 'Diagnosi', 'Anamnesi', 'Motivo ricovero', 'fumo', 'diabete', 'familiarita_cardiovascolare', 'ritmo_all_ingresso', 'esami_all_ingresso', 'esami_alla_dimissione', 'terapia_alla_dimissione']
Score coerenza valori: 0
Entità con valore sbagliato: []
Entità inventate: []

--- Riga 2 ---
Numero di entità totali: 74
Numero di entità trovate: 18
Entità trovate: ['n_cartella', 'data_ingresso_cch', 'data_dimissione_cch', 'nome', 'cognome', 'numero di telefono', 'data_di_nascita', 'Diagnosi', 'Anamnesi', 'Motivo ricovero', 'fumo', 'diabete', 'ipertensione', 'dislipidemia', 'familiarita_cardiovascolare', 'esami_all_ingresso', 'esami_alla_dimissione', 'terapia_alla_dimissione']
Score coerenza valori: 0
Entità con valore sbagliato: []
Entità inventate: []

--- Riga 3 ---
Numero di entità totali:

In [ ]:
show_entity_distribution_multiple(dict_models_df_response)

In [ ]:
show_comparison(dict_models_df_response)

In [ ]:
show_wrong_type_entities(dict_models_df_response)

In [ ]:
show_hallucinated_entities(dict_models_df_response)

In [ ]:
df1 = dict_models_df_response['DeepSeek-V3.pkl']
df2 = dict_models_df_response['LLM_Llama-3.3-70B.pkl']

all_entities = sorted(list(ENTITY_TYPE_MAP.keys()))

fig = go.Figure()

for entity in all_entities:
    # Binario presenza: 1 se valore non nullo/non vuoto/non zero, altrimenti 0
    presence_df1 = df1[entity].apply(lambda x: int(pd.notna(x) and str(x).strip() not in ['0', '', 'nan']))
    presence_df2 = df2[entity].apply(lambda x: int(pd.notna(x) and str(x).strip() not in ['0', '', 'nan']))

    # Violin modello 1 (DeepSeek-V3)
    fig.add_trace(go.Violin(
        x= [entity] * len(presence_df1),  # categoria entità ripetuta per ogni paziente
        y=presence_df1,
        legendgroup=entity,
        scalegroup=entity,
        name="(DeepSeek-V3)",
        side='negative',
        line_color='blue',
        showlegend=(entity == all_entities[0])  # mostra legenda una volta per entità
    ))

    # Violin modello 2 (LLaMA)
    fig.add_trace(go.Violin(
        x=[entity] * len(presence_df2),
        y=presence_df2,
        legendgroup=entity,
        scalegroup=entity,
        name="(LLaMA-3.3)",
        side='positive',
        line_color='orange',
        showlegend=(entity == all_entities[0])
    ))

fig.update_traces(meanline_visible=True)

fig.update_layout(
    title="Distribuzione presenza/assenza entità per paziente (confronto modelli)",
    yaxis_title="Presenza entità (0 = no, 1 = sì)",
    xaxis_title="Entità",
    yaxis=dict(tickmode='array', tickvals=[0, 1]),
    violingap=0,
    violinmode='overlay',
    height=700,
    width=1000
)

fig.show()


In [ ]:
# Controlla se la GPU è disponibile
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Carica tokenizer e modello ClinicalBERT su GPU (se disponibile)
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT").to(device)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Sposta gli input sulla GPU
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    # Mean pooling sui token (escludendo padding)
    attention_mask = inputs['attention_mask']
    token_embeddings = outputs.last_hidden_state  # (batch_size, seq_len, hidden_size)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
    mean_embeddings = sum_embeddings / sum_mask

    return mean_embeddings.squeeze().cpu().numpy()  # Sposta su CPU per numpy

# Lista entità di tipo testo
list_text_entities = [k for k, v in ENTITY_TYPE_MAP.items() if v == 'Text']
print(list_text_entities)

dict_models_df_embeddings = {}
for k, v in dict_models_df_response.items():
    df_embedding = pd.DataFrame()
    for entity in list_text_entities:
        # Pulisce la colonna
        texts = v[entity].fillna('').replace('Nan', '').astype(str)

        # Applica get_embedding a ogni testo
        embeddings = texts.apply(get_embedding)

        # Salva embeddings nel dataframe
        df_embedding[entity] = embeddings

    dict_models_df_embeddings[k] = df_embedding


Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning:


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.



config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

['nome', 'cognome', 'numero di telefono', 'Diagnosi', 'Anamnesi', 'Motivo ricovero', 'Tipo di REDO', 'Terapia', 'esami_all_ingresso', 'Decorso_post_operatorio', 'Tipo_secondo_intervento', 'Causa_II_Run_CEC', 'Causa_morte', 'esami_alla_dimissione', 'terapia_alla_dimissione']


In [ ]:
for entity in list_text_entities:
    vec1 = dict_models_df_embeddings['DeepSeek-V3.pkl'][entity].mean(axis=0).reshape(1, -1)
    vec2 = dict_models_df_embeddings['LLM_Llama-3.3-70B.pkl'][entity].mean(axis=0).reshape(1, -1)

    similarity = cosine_similarity(vec1, vec2)[0][0]

    print(f"Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità '{entity}': {similarity:.4f}\n")


Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'nome': 1.0000

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'cognome': 0.9999

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'numero di telefono': 0.9999

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Diagnosi': 0.9971

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Anamnesi': 0.9955

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Motivo ricovero': 0.9978

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Tipo di REDO': 0.9980

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Terapia': 0.9118

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'esami_all_ingresso': 0.8472

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Decorso_post_operatorio': 0.9979

Similarità tra DeepSeek-V3.pkl e LLM_Llama-3.3-70B.pkl per l'entità 'Tipo_second

In [ ]:
models = ['DeepSeek-V3.pkl', 'LLM_Llama-3.3-70B.pkl']

all_vectors = []
all_labels = []
all_entities = []

for model in models:
    for entity in list_text_entities:
        vectors = dict_models_df_embeddings[model][entity]  # <-- dovrebbe essere (n_tokens, dim)

        for vec in vectors:  # iteriamo su ogni vettore/token
            all_vectors.append(vec)
            all_labels.append(model)
            all_entities.append(entity)

all_vectors = np.array(all_vectors)

# Riduzione dimensionale con t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=30, init='pca', learning_rate='auto')
tsne_result = tsne.fit_transform(all_vectors)

# Creazione DataFrame per il grafico
df_plot = pd.DataFrame({
    'x': tsne_result[:, 0],
    'y': tsne_result[:, 1],
    'model': all_labels,
    'entity': all_entities
})

# Grafico interattivo
fig = px.scatter(
    df_plot, x='x', y='y',
    color='model',
    hover_data=['entity'],
    title='t-SNE projection of all embeddings (token-level)'
)

fig.show()
